In [ ]:
from __future__ import annotations
import itertools
import pickle

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sstt

from online_semg_posture_adaptation import dataset as ds

In [ ]:
# results file of each experiment
fullpath_results_multi = './results/results_multitrain.pkl'
fullpath_results_calib = './results/results_adaptation.pkl'

# load all results files

with open(fullpath_results_multi, 'rb') as f:
    results_multi = pickle.load(f)['results']

with open(fullpath_results_calib, 'rb') as f:
    results_calib = pickle.load(f)['results']

In [ ]:
FULL_CARTPROD = itertools.product(
    range(ds.NUM_SUBJECTS),
    range(ds.NUM_DAYS),
    range(ds.NUM_POSTURES),  # reference posture
    range(ds.NUM_POSTURES),  # arrival posture
)
FULL_CARTPROD = list(FULL_CARTPROD)  # for reuse, notebook-style

subject_sublist     = [0, 1, 2, 3, 4, 5, 6]
day_sublist         = [0]  # [0, 1, 2, 3, 4, 5, 6, 7]
ref_posture_sublist = [0, 1, 2, 3]  # reference postures
tgt_posture_sublist = [0, 1, 2, 3]  # arrival postures

# CUSTOM_CARTPROD = FULL_CARTPROD
CUSTOM_CARTPROD_BASELINE = itertools.product(
    subject_sublist,
    day_sublist,
    tgt_posture_sublist,
)
CUSTOM_CARTPROD_BASELINE = list(CUSTOM_CARTPROD_BASELINE)  # for reuse, notebook-style

CUSTOM_CARTPROD = itertools.product(
    subject_sublist,
    day_sublist,
    ref_posture_sublist,
    tgt_posture_sublist,
)
CUSTOM_CARTPROD = list(CUSTOM_CARTPROD)  # for reuse, notebook-style

In [ ]:
# Format accuracy data


# multi-posture training
acc_multi = np.zeros((ds.NUM_SUBJECTS, ds.NUM_DAYS, ds.NUM_POSTURES), dtype=np.float32)

for idx_subject, idx_day, idx_posture in itertools.product(
    range(ds.NUM_SUBJECTS), range(ds.NUM_DAYS), range(ds.NUM_POSTURES),
):
    acc_multi[idx_subject, idx_day, idx_posture] = \
        results_multi['subject'][idx_subject]['day'][idx_day]['posture'][idx_posture]['validation']['accuracy']

custom_mask_mesh = np.ix_(subject_sublist, day_sublist, tgt_posture_sublist)
acc_multi = acc_multi[custom_mask_mesh]  # select all dimensions simultaneously


# calibration modes
acc_calib_none = np.zeros((ds.NUM_SUBJECTS, ds.NUM_DAYS, ds.NUM_POSTURES, ds.NUM_POSTURES), dtype=np.float32)
acc_calib_pcaon = np.zeros((ds.NUM_SUBJECTS, ds.NUM_DAYS, ds.NUM_POSTURES, ds.NUM_POSTURES), dtype=np.float32)

for idx_subject, idx_day, idx_ref_posture, idx_tgt_posture in CUSTOM_CARTPROD:

    # no adaptation
    acc_calib_none[idx_subject, idx_day, idx_ref_posture, idx_tgt_posture] = \
        results_calib['subject'][idx_subject]['day'][idx_day]['reference_posture'][idx_ref_posture][
            'target_posture'][idx_tgt_posture]['recalibration_mode']['none']['validation']['accuracy']

    # online PCA
    acc_calib_pcaon[idx_subject, idx_day, idx_ref_posture, idx_tgt_posture] = \
        results_calib['subject'][idx_subject]['day'][idx_day]['reference_posture'][idx_ref_posture][
            'target_posture'][idx_tgt_posture]['recalibration_mode']['pca_online']['validation']['accuracy']


# calibration modes: discard the posture-diagonal ones
acc_calib_none_flat = []
acc_calib_pcaon_flat = []

for idx_subject, idx_day, idx_ref_posture, idx_tgt_posture in CUSTOM_CARTPROD:
    if idx_tgt_posture != idx_ref_posture:
        acc_calib_none_flat.append(acc_calib_none[idx_subject, idx_day, idx_ref_posture, idx_tgt_posture])
        acc_calib_pcaon_flat.append(acc_calib_pcaon[idx_subject, idx_day, idx_ref_posture, idx_tgt_posture])

acc_calib_none_flat = np.array(acc_calib_none_flat, dtype=np.float32)
acc_calib_pcaon_flat = np.array(acc_calib_pcaon_flat, dtype=np.float32)

In [ ]:
# Statistics: median and Median Absolute Deviation (MAD)

# median
acc_multi_med = np.median(acc_multi)
acc_calib_none_med = np.median(acc_calib_none_flat)
acc_calib_pcaon_med = np.median(acc_calib_pcaon_flat)

# mad
acc_multi_mad = sstt.median_abs_deviation(acc_multi, axis=(0, 1, 2))
acc_calib_none_mad = sstt.median_abs_deviation(acc_calib_none_flat)
acc_calib_pcaon_mad = sstt.median_abs_deviation(acc_calib_pcaon_flat)


# Display results
print(
    f"\t\t\tmedian +/- mad\n"
    f"---------------------------------------------------------------------------------------------------------\n"
    f"Multiposture train:\t{acc_multi_med:.4f} +/- {acc_multi_mad:.4f}\n"
    f"---------------------------------------------------------------------------------------------------------\n"
    f"No adaptation:\t\t{acc_calib_none_med:.4f} +/- {acc_calib_none_mad:.4f}\n"
    f"Online PCA:\t\t{acc_calib_pcaon_med:.4f} +/- {acc_calib_pcaon_mad:.4f}\n"
)

In [ ]:
recovered_drop_med_fraction = (acc_calib_pcaon_med - acc_calib_none_med) / (acc_multi_med - acc_calib_none_med)
print(recovered_drop_med_fraction.round(4))